In [1]:
import nltk
import numpy as np
import pandas as pd

from collections import deque
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
from sklearn.metrics import confusion_matrix
from IPython.display import display

In [2]:
# Load vectors directly from the file
word_embeddings = KeyedVectors.load_word2vec_format('./data/news_word_embeddings.bin', binary = True)

In [3]:
nltk.download('treebank', quiet = True)
nltk.download('tagsets', quiet = True)
nltk.download('stopwords', quiet = True)

True

In [4]:
def extract_data(ratio = 0.9):
    assert (ratio >= 0 and ratio <= 1.0)
    
    # treebank corpus
    file_names = nltk.corpus.treebank.fileids()
    
    # extract tagged words
    tagged_words = []
    for file_name in file_names:
        tagged_words += list(nltk.corpus.treebank.tagged_words(file_name))
        
    # data offset
    offset = int(len(tagged_words) * ratio)
        
    return (tagged_words[:offset], tagged_words[offset:])

In [5]:
train_data, valid_data = extract_data()

In [6]:
class Base:
    """ Base model class """
    
    def __init__(self):
        pass

In [7]:
class HMM:
    """ HMM model based on Search Beam & Viterbi algorithm """
    
    max_suffix_size: int = 6
    max_prefix_size: int = 4
    
    def __init__(self, data):
        
        self.train_data = data
        
        # initialize the model
        self.init(data)
    
    def init(self, data):
                
        # unpack word tokens and pos labels
        tokens, labels = zip(*data)
        
        # our vocabulary
        self.vocabulary, self.labels = set(tokens), set(labels)
        self.label_types = list(self.labels)
        
        # freq token <-> pos label
        self.freq_dist = nltk.FreqDist(data)
        self.freq_tokens = {}
        
        # resolve uppercase case
        for (token, tag), freq in self.freq_dist.items():
            self.freq_tokens[(token, tag)] = freq
            if token.islower():
                key = (token.capitalize(), tag)
            else:
                key = (token.lower(), tag)
                
            if key not in self.freq_tokens:
                self.freq_tokens[key] = freq
                
        # generate token bounds
        self.token_bounds = self.get_token_bounds()
        
        # create 2-gram generator
        grams = nltk.bigrams(labels)

        # create 1-gram freq and 2-gram freq for labels
        self.freq_gram_labels = (nltk.FreqDist(labels), nltk.FreqDist(grams))
        
    def get_token_bounds(self, freq_trim = 10):
        
        # filter high-freq words (articles, prepositions)
        freq_word_tags = { k: v for k, v in self.freq_tokens.items() if v < freq_trim }
        
        token_prefixes, token_suffixes = {}, {}
        for (token, tag), v in self.freq_tokens.items():
            size, index = len(token), self.label_types.index(tag)
            
            # counting prefixes
            prefix_end = min(size - 1, self.max_prefix_size)
            for i in range(1, prefix_end):
                prefix = token[:i]
                
                if prefix not in token_prefixes:
                    token_prefixes[prefix] = {
                        'freq': np.zeros((len(self.label_types),)),
                        'total': 0
                    }
                    
                token_prefixes[prefix]['freq'][index] += v
                token_prefixes[prefix]['total'] += v
            
            # counting suffixes
            suffix_start = max(1, size - 1 - self.max_suffix_size)
            for i in range(size - 1, suffix_start, -1):
                suffix = token[i:]
                
                if suffix not in token_suffixes:
                    token_suffixes[suffix] = {
                        'freq': np.zeros((len(self.label_types),)),
                        'total': 0
                    }
                    
                token_suffixes[suffix]['freq'][index] += v
                token_suffixes[suffix]['total'] += v
                
        return (token_suffixes, token_prefixes)
    
    def get_bound_prob(self, token, tag):
        
        prefixes_prob, suffixes_prob = [], []
        size, index = len(token), self.label_types.index(tag)
        
        # compute prob suffix
        suffix_start = max(1, size - 1 - self.max_suffix_size)
        for i in range(size - 1, suffix_start, -1):
            suffix = token[i:]
            
            if suffix in self.token_bounds[0]:
                freq_bound = self.token_bounds[0][suffix]['freq'][index]
                if suffix in self.token_bounds[0] and freq_bound != 0:
                    suffixes_prob.append(freq_bound / self.token_bounds[0][suffix]['total'])
                
        # compute prob prefix
        prefix_end = min(size - 1, self.max_prefix_size)
        for i in range(1, prefix_end):
            prefix = token[:i]
            
            if prefix in self.token_bounds[1]:
                freq_bound = self.token_bounds[1][prefix]['freq'][index]
                if prefix in self.token_bounds[1] and freq_bound != 0:
                    prefixes_prob.append(freq_bound / self.token_bounds[1][prefix]['total'])
                
        # prefixes
        weighted_left_prob = np.dot(prefixes_prob, prefixes_prob)
        
        # sufixes
        weighted_right_prob = np.dot(suffixes_prob, suffixes_prob)
                
        return weighted_left_prob * weighted_right_prob
        
    def get_seq_prob(self, word, prev_tag, tag):
        
        # case if the target word is unknown
        word_likelihood = None if word not in self.vocabulary else self.freq_tokens.get((word, tag), 0)
        
        if word_likelihood is None:
            word_likelihood = self.get_bound_prob(word, tag)
            
        emission = word_likelihood / self.freq_gram_labels[0][tag]
        transition = self.freq_gram_labels[1][(prev_tag, tag)] / self.freq_gram_labels[0][prev_tag]
        
        return emission * transition
        
    def forward(self, features, window_size = None):
        
        # avoid prob underflow
        if window_size is None:
            window_size = len(features)
        
        # sequence size
        seq_size, label_size = len(features), len(self.label_types)
        
        # memoization of max prob for each t timestamp and indexing for backtracking
        T1, T2 = np.zeros((window_size, label_size)), np.zeros((window_size, label_size))
        
        # initialize starting tag with maximum likelihood
        token = features[0]
        indices = [ self.freq_tokens.get((token, tag), 0) for tag in self.label_types ]
        T1[0][int(np.argmax(indices))] = 1.0
        
        # initialize helper variables
        labels, offset = [], 0
        
        while True:
            
            # case of overflow
            offset_lim = min(offset + window_size, seq_size) - offset
            
            # forward pass
            for t in range(1, offset_lim):
                
                # current target token
                token = features[t + offset]

                # markov assumption
                for index, label in enumerate(self.label_types):
                    for prev_index, prev_label in enumerate(self.label_types):
                        prob = self.get_seq_prob(token, prev_label, label) * T1[t - 1][prev_index]

                        if prob > T1[t][index]:
                            T1[t][index] = prob
                            T2[t][index] = prev_index

            # backward pass
            prob_max_index = np.argmax(T1[offset_lim - 1, :])
            
            labels_window, prob_index = deque(), prob_max_index
            for t in range(offset_lim, 1, -1):
                prob_index = int(T2[t - 1][prob_index])
                
                labels_window.appendleft(self.label_types[prob_index])
            
            # reset mem
            T1.fill(0)
            T2.fill(0)
            
            # assign max prob to last word
            T1[0][prob_max_index] = 1.0
            
            offset += window_size - 1
            labels += list(labels_window)
            
            if offset >= seq_size:
                break
        
        # update with last item
        labels.append(self.label_types[prob_max_index])
            
        return labels  

In [8]:
class MEMM:
    """ MEMM model based on Search Beam & Viterbi algorithm """
    
    def __init__(self, data):
        
        self.train_data = data
        
        # initialize the model
        self.init(data)
    
    def init(self, data):
                
        # unpack word tokens and pos labels
        tokens, labels = zip(*data)
        
        # our vocabulary
        self.vocabulary, self.labels = set(tokens), set(labels)
        self.label_types = list(self.labels)
        
        # create train features
        inputs = []
        for index, (token, tag) in enumerate(zip(tokens, labels)):
            inputs.append(self.get_feature(tokens, labels, index))

        # create train targets
        targets = [ self.label_types.index(label) for label in labels ]
        
        # freq token <-> pos label
        self.model = LogisticRegression(random_state = 0, max_iter = 300, solver = 'lbfgs', multi_class = 'multinomial').fit(inputs, targets)
    
    def get_feature(self, tokens, labels, index, window_size = 3, bare = False):
        
        # features
        features = []
        
        # boundary indices
        index_start, index_end = index - window_size, index + window_size

        for i in range(index_start, index_end):

            # current token
            t = tokens[i] if i >= 0 and i < len(tokens) else None
            
            # get token vector representation
            token_vec = word_embeddings[t] if t is not None and t in word_embeddings else np.zeros((word_embeddings.vector_size,))
            
            features.append(token_vec)
            
        if bare:
            index_start, index = 0, len(labels)
        
        for i in range(index_start, index - 1):
            
            # current tag
            t = labels[i] if i >= 0 and i < len(labels) else None
            
            if t == -1:
                features.append([-1])
            else:
                features.append([self.label_types.index(t) if t is not None else -1])

        # concatenate features
        return np.concatenate(features)
        
    def forward(self, features, window_size = None):
        
        # avoid prob underflow
        if window_size is None:
            window_size = len(features)
        
        # sequence size
        seq_size, label_size = len(features), len(self.label_types)
        
        # memoization of max prob for each t timestamp and indexing for backtracking
        T1, T2 = np.zeros((window_size, label_size)), np.zeros((window_size, label_size))
        
        # initialize with equal prob
        T1[0] = [ 1 / label_size ] * label_size
        
        # initialize helper variables
        labels, offset = [], 0
        
        while True:
            
            # case of overflow
            offset_lim = min(offset + window_size, seq_size) - offset
            
            # forward pass
            for t in range(1, offset_lim):
                
                for prev_index, prev_tag in enumerate(self.label_types):
                    
                    # min_start index
                    min_start, min_offset = max(0, t - 3), 0 if t - 3 >= 0 else t - 3

                    # previous taggings based on maximum likelihood
                    taggings, sample_index = deque(), int(prev_index)
                    
                    taggings.appendleft(self.label_types[sample_index])
                    for i in range(t - 1, min_start, -1):
                        sample_index = int(T2[i][sample_index])
                        
                        taggings.appendleft(self.label_types[sample_index])
                        
                    if min_offset < 0:
                        taggings = labels[min_offset:] + list(taggings)
                        
                    if len(taggings) < 3:
                        taggings = [-1] * (3 - len(taggings)) + taggings
                    
                    # features
                    X = np.expand_dims(self.get_feature(features, taggings, t + offset, bare = True), 0)
                    
                    # make prediction (len(self.label_types), )
                    prob = self.model.predict_proba(X)[0]
                    
                    # joint probability with previous assumptions
                    prob *= T1[t - 1][prev_index]
                    
                    for index in range(len(self.label_types)):
                        if prob[index] > T1[t][index]:
                            T1[t][index] = prob[index]
                            T2[t][index] = prev_index

            # backward pass
            prob_max_index = np.argmax(T1[-1, :])
            
            labels_window, prob_index = deque(), prob_max_index
            for t in range(offset_lim, 1, -1):
                prob_index = int(T2[t - 1][prob_index])
                
                labels_window.appendleft(self.label_types[prob_index])
            
            # reset mem
            T1.fill(0)
            T2.fill(0)
            
            # assign max prob to last word
            T1[0][prob_max_index] = 1.0
            
            offset += window_size - 1
            
            # concatenate our outputs
            labels += list(labels_window)
            
            if offset >= seq_size:
                break
        
        # update with last item
        labels.append(self.label_types[prob_max_index])
            
        return labels

In [9]:
def validate_model(network, valid_data, window_size = 7):

    # pre-process labelled data 
    features, targets = zip(*valid_data)
    
    # generate network outputs
    outputs = network.forward(features, window_size)
    
    # compute number of valid labels
    labels_predicted = 0
    for predicted, target in zip(outputs, targets):
        labels_predicted += predicted == target
    
    # building the confusion matrix
    mat = confusion_matrix(targets, outputs, labels = network.label_types)
    
    # size of annotations
    size = len(targets)
    
    return labels_predicted / size, (labels_predicted, size), mat

In [10]:
def test_model(model_type, train_data, valid_data, n = 6):
    
    # reference model definition
    model = HMM if model_type == 'hmm' else MEMM
    network = model(train_data)
    
    # validate the model
    score, occ, mat = validate_model(network, valid_data)
    print(f"Model {model_type} -> {score}")
    
    # labels size (tags)
    size = len(network.label_types)
    
    # remove positive true
    mat = ~(np.identity(size) == 1) * mat
    
    # building the data frame with corresponding labels and extracting the n largest NT
    mat = pd.DataFrame(mat, index = network.label_types, columns = network.label_types)
    mat_largest_n = mat.unstack().nlargest(n)
    
    display(mat_largest_n)

In [11]:
for model_type in ['hmm', 'memm']:
    test_model(model_type, train_data, valid_data)

Model hmm -> 0.9229241160111243


NN   NNP    85
NNP  NN     73
NN   JJ     39
VBN  VBD    31
NN   VBG    27
JJ   NN     26
dtype: int64

C:\Users\DomainFlag\Miniconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Model memm -> 0.849523241954708


,   IN        70
    CC        48
CC  IN        39
    ,         30
,   .         28
    -NONE-    28
dtype: int64

In [12]:
# create an instance of our model
network = HMM(train_data)

In [13]:
seqs = [
    'The weather is pretty much ok for sightseeing',
    'I said her that her lines are well written',
    'Tea is much better than coffe',
    'Like it or not, we did a good job'
]

In [14]:
for s in seqs:
    output = network.forward(s.split(' '))
    
    print(f"{s} -> {output}")

The weather is pretty much ok for sightseeing -> ['RP', 'RP', 'RP', 'RP', 'RP', 'RP', 'RP', 'RP']
I said her that her lines are well written -> ['PRP', 'VBD', 'PRP', 'IN', 'PRP$', 'NNS', 'VBP', 'RB', 'VBN']
Tea is much better than coffe -> ['NN', 'VBZ', 'RB', 'JJR', 'IN', 'NNP']
Like it or not, we did a good job -> ['RP', 'RP', 'RP', 'RP', 'RP', 'RP', 'RP', 'RP', 'RP']
